In [2]:
!pip install fastapi uvicorn pymongo python-jose[cryptography] bcrypt pyngrok nest_asyncio


In [4]:
!pip install python-multipart


In [5]:
# main.py

from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from jose import jwt, JWTError
from pydantic import BaseModel
from pymongo import MongoClient
import bcrypt
import uvicorn
import nest_asyncio

SECRET_KEY = "mysecretkey"
ALGORITHM = "HS256"

client = MongoClient("mongodb://localhost:27017")
db = client["propertyDB"]
users = db["users"]
properties = db["properties"]

app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

class User(BaseModel):
    email: str
    password: str

class Property(BaseModel):
    title: str
    description: str
    price: float
    location: str
    type: str
    bedrooms: int
    bathrooms: int
    size: float
    amenities: list

def create_token(data: dict):
    return jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)

def verify_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload.get("sub")
    except JWTError:
        return None

def get_current_user(token: str = Depends(oauth2_scheme)):
    user_email = verify_token(token)
    if not user_email:
        raise HTTPException(status_code=401, detail="Invalid token")
    return user_email

@app.post("/register")
def register(user: User):
    if users.find_one({"email": user.email}):
        raise HTTPException(status_code=400, detail="User exists")
    hashed_pw = bcrypt.hashpw(user.password.encode(), bcrypt.gensalt())
    users.insert_one({"email": user.email, "password": hashed_pw})
    return {"msg": "User created"}

@app.post("/token")
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    user = users.find_one({"email": form_data.username})
    if not user or not bcrypt.checkpw(form_data.password.encode(), user["password"]):
        raise HTTPException(status_code=401, detail="Invalid credentials")
    token = create_token({"sub": user["email"]})
    return {"access_token": token, "token_type": "bearer"}

@app.post("/properties")
def add_property(prop: Property, user_email: str = Depends(get_current_user)):
    properties.insert_one({**prop.dict(), "createdBy": user_email})
    return {"msg": "Property added"}

@app.get("/properties")
def get_properties():
    return list(properties.find({}, {"_id": 0}))

@app.delete("/properties/{title}")
def delete_property(title: str, user_email: str = Depends(get_current_user)):
    result = properties.delete_one({"title": title, "createdBy": user_email})
    if result.deleted_count == 0:
        raise HTTPException(status_code=403, detail="Unauthorized or not found")
    return {"msg": "Deleted"}


In [6]:
!ngrok config add-authtoken 2xl3YRPjsjGLChy4ahmjS6kOQ48_79sF8PqdRo99G7mmLQtd1

                                                                                                    
Installing ngrok ... 
                                                                                                    
Authtoken saved to configuration file: C:\Users\DELL\AppData\Local/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

uvicorn.run(app, host="0.0.0.0", port=8000)
